In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplt
import matplotlib as plt
import datetime as dati

In [2]:
dataset = pd.read_csv("./entrenamieto.csv", parse_dates = ['Account_Created_Date', 'Opportunity_Created_Date',
                                                           'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
                                                           'Quote_Expiry_Date', 'Last_Modified_Date','Last_Activity' ])
# Limpio registros con muchos datos faltantes o inutiles
dataset = dataset.drop(columns= ['Prod_Category_A','Product_Category_B','Actual_Delivery_Date','Submitted_for_Approval','Last_Activity','Price','Currency','Brand','Size','Product_Type'])
dataset.dtypes

ID                                             int64
Region                                        object
Territory                                     object
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Bureaucratic_Code                             object
Account_Created_Date                  datetime64[ns]
Source                                        object
Billing_Country                               object
Account_Name                                  object
Opportunity_Name                              object
Opportunity_ID                                 int64
Sales_Contract_No                             object
Account_Owner                                 object
Opportunity_Owner                             object
Account_Type                                  object
Opportunity_Type                              

In [3]:
#limpio los datos no finalzizados
dataset = dataset.loc[(dataset['Stage']== 'Closed Won') | (dataset['Stage']== 'Closed Lost')]
dataset['Stage'].value_counts()

Closed Won     9533
Closed Lost    7350
Name: Stage, dtype: int64

# Convertir mondeas

In [4]:
dataset['conversion'] = dataset['ASP_(converted)']/dataset['ASP']

cotizacion = dataset.groupby(['ASP_Currency', 'Opportunity_Created_Date']).agg({'conversion': 'mean'}).reset_index()
#print(dataset[dataset['Stage']== 'Closed Won']['conversion'].value_counts())
cotizacion = cotizacion.rename(columns = {'conversion': 'conversion_diaria'})
cotizacion

dataset= dataset.merge(cotizacion, on = ['ASP_Currency', 'Opportunity_Created_Date'], how = 'left')

dataset

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,conversion,conversion_diaria
0,27761,EMEA,None,1,1,1,1,Bureaucratic_Code_4,2015-06-16,None,...,Q2,2016,10,EUR,5272800.00,EUR,5272800.0,Closed Lost,1.131096,1.131095
1,27760,EMEA,None,0,0,0,0,Bureaucratic_Code_4,2015-06-16,None,...,Q1,2016,0,EUR,48230.00,EUR,48230.0,Closed Won,1.131094,1.131095
2,27446,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,Q1,2016,0,USD,83865.60,USD,83865.6,Closed Won,1.000000,1.000000
3,16808,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,Q1,2018,14,USD,7421881.50,USD,7421881.5,Closed Lost,1.000000,1.000000
4,16805,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,Q1,2018,25,USD,13357192.50,USD,13357192.5,Closed Lost,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16878,8781,EMEA,Austria,1,1,1,1,Bureaucratic_Code_4,2016-01-15,Source_7,...,Q1,2016,0,EUR,103350.00,EUR,299715.0,Closed Won,1.131096,1.131093
16879,8786,EMEA,Austria,1,1,1,1,Bureaucratic_Code_4,2016-01-15,Source_7,...,Q2,2016,0,EUR,93015.00,EUR,299715.0,Closed Won,1.131096,1.131093
16880,8792,EMEA,Austria,1,1,1,1,Bureaucratic_Code_4,2016-01-15,Source_7,...,Q1,2016,0,EUR,103350.00,EUR,299715.0,Closed Won,1.131096,1.131093
16881,28561,Americas,NE America,1,1,1,1,Bureaucratic_Code_4,2015-10-20,None,...,Q2,2016,4,USD,2346796.88,USD,0.0,Closed Lost,1.000000,1.000000


In [5]:
dataset['conversion'] = dataset.loc[:,'conversion'].fillna(dataset['conversion_diaria'])

In [6]:
dataset = dataset.drop(dataset[dataset['conversion'] == np.inf].index)

In [7]:
dataset.loc[dataset['ASP_Currency'] == 'USD', 'conversion'] = 1
dataset.loc[dataset.ASP_Currency == 'JPY' , 'conversion']= dataset.loc[dataset.ASP_Currency == 'JPY' , 'conversion'].fillna( dataset.loc[dataset.ASP_Currency == 'JPY', 'conversion'].mean())
dataset.loc[dataset.ASP_Currency == 'EUR' , 'conversion']= dataset.loc[dataset.ASP_Currency == 'EUR' , 'conversion'].fillna( dataset.loc[dataset.ASP_Currency == 'EUR', 'conversion'].mean())
dataset['Total_Amount_converted'] = dataset['Total_Amount'] * dataset['conversion']

# diferencias de tiempo

In [8]:
dataset['delta_planed'] = dataset['Planned_Delivery_End_Date'] - dataset['Planned_Delivery_Start_Date']

In [9]:
dataset.dtypes

ID                                              int64
Region                                         object
Territory                                      object
Pricing, Delivery_Terms_Quote_Appr              int64
Pricing, Delivery_Terms_Approved                int64
Bureaucratic_Code_0_Approval                    int64
Bureaucratic_Code_0_Approved                    int64
Bureaucratic_Code                              object
Account_Created_Date                   datetime64[ns]
Source                                         object
Billing_Country                                object
Account_Name                                   object
Opportunity_Name                               object
Opportunity_ID                                  int64
Sales_Contract_No                              object
Account_Owner                                  object
Opportunity_Owner                              object
Account_Type                                   object
Opportunity_Type            

In [10]:
dataset['acount_age'] = dataset['Opportunity_Created_Date'] - dataset['Account_Created_Date']

# features por oportunity ID

In [11]:
#Opportunity_ID
features= ['sum','mean','median','std', 'max', 'min']

dataset_numerico = dataset[['Opportunity_ID','ASP', 'ASP_(converted)', 'TRF', 'Total_Amount', 'Total_Taxable_Amount', 'Total_Amount_converted']]
#ASP                                           float64
#ASP_(converted)                               float64
#TRF                                             int64
#Total_Amount                                  float64
#Total_Taxable_Amount                          float64
#Total_Amount_converted                        float64

features_num= dataset_numerico.groupby('Opportunity_ID').agg(features)
level_0 = features_num.columns.get_level_values(0)
level_1 = features_num.columns.get_level_values(1)
features_num.columns = level_0 + '_' + level_1
features_num.reset_index()
dataset = dataset.join(features_num, on= 'Opportunity_ID', how = 'left')

In [12]:
dataset.dtypes

ID                                      int64
Region                                 object
Territory                              object
Pricing, Delivery_Terms_Quote_Appr      int64
Pricing, Delivery_Terms_Approved        int64
                                       ...   
Total_Amount_converted_mean           float64
Total_Amount_converted_median         float64
Total_Amount_converted_std            float64
Total_Amount_converted_max            float64
Total_Amount_converted_min            float64
Length: 83, dtype: object

# Gererador de features cumulative para categoricas

In [13]:
dataset['ganado']= (dataset['Stage']== 'Closed Won' )
dataset['counter']= 1
categorical = ['Region', 'Territory', 'Product_Name', 'Account_Owner','Bureaucratic_Code','Billing_Country', 'Account_Name', 'Opportunity_Owner','Quote_Type', 'Delivery_Terms','Last_Modified_By', 'Product_Family', 'Month', 'Delivery_Quarter']
for agrupador in categorical:
    gruped_product = dataset.groupby(agrupador)[['ganado','counter']].cumsum()
    gruped_product[agrupador + '_mean'] = gruped_product['ganado']/ gruped_product['counter']
    gruped_product= gruped_product.rename(columns = {'ganado':(agrupador + '_ganados'), 'counter': (agrupador + '_count') })
    dataset= dataset.join(gruped_product)

    

In [14]:
dataset.dtypes

ID                                      int64
Region                                 object
Territory                              object
Pricing, Delivery_Terms_Quote_Appr      int64
Pricing, Delivery_Terms_Approved        int64
                                       ...   
Month_count                             int64
Month_mean                            float64
Delivery_Quarter_ganados                int64
Delivery_Quarter_count                  int64
Delivery_Quarter_mean                 float64
Length: 127, dtype: object

# categorical de a pares

In [15]:
for i in range(len(categorical) -1):
    for j in range(i +1, len(categorical)):
        gruped_product = dataset.groupby([categorical[i],categorical[j]])[['ganado','counter']].cumsum()
        gruped_product[categorical[i] + '_'+ categorical[j] + '_mean'] = gruped_product['ganado']/ gruped_product['counter']
        gruped_product= gruped_product.rename(columns = {'ganado':(categorical[i] + '_'+ categorical[j] + '_ganados'), 'counter': (categorical[i] + '_'+ categorical[j] + '_count') })
        dataset= dataset.join(gruped_product)
        

In [16]:
dataset.dtypes

ID                                         int64
Region                                    object
Territory                                 object
Pricing, Delivery_Terms_Quote_Appr         int64
Pricing, Delivery_Terms_Approved           int64
                                          ...   
Product_Family_Delivery_Quarter_count      int64
Product_Family_Delivery_Quarter_mean     float64
Month_Delivery_Quarter_ganados             int64
Month_Delivery_Quarter_count               int64
Month_Delivery_Quarter_mean              float64
Length: 400, dtype: object